In [0]:
# 1. Création d’une table Delta et insertion initiale
from pyspark.sql import SparkSession

# Création d'une session Spark (dans Databricks, elle est déjà disponible)
spark = SparkSession.builder.appName("DeltaACIDExample").getOrCreate()

# Chemin de la table Delta
delta_table_path = "/tmp/delta_table_acid"

# Création de la table Delta avec des données initiales
data = [
    (1, "Produit A", 100),
    (2, "Produit B", 200)
]
columns = ["id", "product", "price"]

df = spark.createDataFrame(data, columns)

# Écriture initiale des données dans une table Delta
df.write.format("delta").mode("overwrite").save(delta_table_path)

print("Table Delta initialisée avec les données suivantes :")
df.show()


In [0]:
# 2. Ajout de données dans la table Delta
# Ajout de nouvelles données dans la table Delta
new_data = [
    (3, "Produit C", 300),
    (4, "Produit D", 400)
]

new_df = spark.createDataFrame(new_data, columns)

# Écriture des nouvelles données dans la table Delta
new_df.write.format("delta").mode("append").save(delta_table_path)

print("Données ajoutées à la table Delta :")
spark.read.format("delta").load(delta_table_path).show()


In [0]:
# 3. Mise à jour des données existantes
# Mise à jour du prix du produit avec id=1
from delta.tables import DeltaTable

# Charger la table Delta
delta_table = DeltaTable.forPath(spark, delta_table_path)

# Appliquer une mise à jour
delta_table.update(
    condition="id = 1",  # Condition pour identifier la ligne à mettre à jour
    set={"price": "150"}  # Mettre à jour la colonne price
)

print("Table Delta après la mise à jour :")
delta_table.toDF().show()


In [0]:
# 4. Gestion des conflits entre plusieurs utilisateurs
'''
Ce scénario simule un conflit lorsque deux utilisateurs tentent de modifier les mêmes données simultanément. Delta Lake garantit que les transactions sont gérées de manière atomique.
'''

# Transaction simulée par deux utilisateurs

# Utilisateur 1 : Mise à jour du produit avec id=2
delta_table.update(
    condition="id = 2",
    set={"price": "250"}
)

# Utilisateur 2 : Tentative d'ajout d'une ligne avec id=2 (conflit potentiel)
try:
    conflicting_data = [(2, "Produit B - Modifié", 300)]
    conflicting_df = spark.createDataFrame(conflicting_data, columns)

    # Écrire des données conflictuelles
    conflicting_df.write.format("delta").mode("append").save(delta_table_path)
except Exception as e:
    print("Conflit détecté lors de l'écriture concurrente :")
    print(e)

# Lecture finale de la table Delta
print("État final de la table Delta :")
delta_table.toDF().show()
